In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%pip install wandb
!wandb

Usage: wandb [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  agent         Run the W&B agent
  artifact      Commands for interacting with...
  controller    Run the W&B local sweep...
  disabled      Disable W&B.
  docker        W&B docker lets you run your...
  docker-run    Simple wrapper for `docker run`...
  enabled       Enable W&B.
  init          Configure a directory with...
  launch        Launch or queue a job from a
                uri...

  launch-agent  Run a W&B launch agent...
  local         Launch local W&B container...
  login         Login to Weights & Biases
  offline       Disable W&B sync
  online        Enable W&B sync
  pull          Pull files from Weights & Biases
  restore       Restore code, config and docker...
  status        Show configuration settings
  sweep         Create a sweep
  sync          Upload an offline training...
  verify        Verify your local instance


In [10]:
%pip install kaggle --upgrade
import os
fpath = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
if not os.path.isfile(fpath):
    try:
        os.mkdir(os.path.join(os.path.expanduser("~"), ".kaggle"))
    except Exception:
        pass
    with open(fpath, "w") as f:
        f.write(input(f"input {fpath}"))
    !chmod 600 {fpath}

In [11]:
if 'datasets' not in os.listdir(os.getcwd()) or 'animals10' not in os.listdir(os.path.join(os.getcwd(), 'datasets')):
    !kaggle datasets download alessiocorrado99/animals10
    !mkdir datasets
    !unzip animals10.zip -d datasets/animals10
    !rm animals10.zip

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class someNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(IM_HEIGHT*IM_WIDTH*3, 500)
        self.l2 = nn.Linear(500, 10)
    def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        x = self.l2(x)
        x = F.log_softmax(x)
        return x


In [13]:
from tqdm import tqdm
def train(model, dev, train_set_loader, test_set_loader, optimizer, epoch, WAND, LOG_INTERVAL, VAL_INTERVAL):
    model.train()
    for bI, (x_b, y_b) in enumerate(tqdm(train_set_loader)):
        x_b = review(x_b)
        x_b, y_b = preprocess(x_b, y_b)
        pred = model.forward(x_b)
        loss = loss_fn(pred, y_b)
        # clear gradients for this training step
        optimizer.zero_grad()
        # backprop, compute gradients
        loss.backward()
        # apply gradients
        optimizer.step()
        print(f"loss: {loss}")
        if bI % LOG_INTERVAL == 0:
            loss_final = (loss/x_b.shape[0])
            if WAND: wandb.log({"train_loss": loss_final})
            print(bI,loss_final)
        if bI % VAL_INTERVAL == 0:
            test(model, dev, test_set_loader, WAND)

def test(model, dev, test_set_loader, WAND):
    with torch.no_grad():
        val_loss = torch.Tensor(len(test_set_loader)).to(dev)
        for i, (x_v_b, y_v_b) in enumerate(tqdm(test_set_loader)):
            x_v_b, y_v_b = preprocess(x_v_b, y_v_b)
            x_v_b = review(x_v_b)
            pred = model.forward(x_v_b)
            top_1_pred = torch.argmax(pred)
            top_1_accuracy = (top_1_pred == y_v_b).sum()/y_v_b.shape[0]
            loss = loss_fn(pred, y_v_b)
            if WAND:
                loss_final = loss/y_v_b.shape[0]
                wandb.log({"test_loss": loss_final})
                wandb.log({"top_1_accuracy": top_1_accuracy})
            print(f"i: {i}, loss_final: {loss_final}, top_1_acc: {top_1_accuracy}")


In [14]:
WAND = True
LOG_INTERVAL = 1
N_EPOCH = 1
LR = 0.001
BS = 64
VAL_INTERVAL = int(torch.ceil(torch.tensor((25_000*0.75)/(BS*10))).item())
print(VAL_INTERVAL)
IM_HEIGHT = 425
IM_WIDTH = 600
TRAIN_DAT_PERC = 0.75

30


In [15]:
if WAND:
  import wandb
  wandb_run = wandb.init(project="animals10", entity="0xasim")
  wandb.config = {
    "learning_rate": LR,
    "epochs": N_EPOCH,
    "batch_size": BS
  }
  run_name = wandb_run.name

wandb: Currently logged in as: 0xasim. Use `wandb login --relogin` to force relogin


In [16]:
import torch
import os
# torch.manual_seed(17)
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.Resize([IM_HEIGHT, IM_WIDTH]), transforms.ToTensor()])
path = os.path.join(os.getcwd(), "datasets", "animals10", "raw-img")
dataset = datasets.ImageFolder(path, transform=transform)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*TRAIN_DAT_PERC), int(len(dataset)*round(1-TRAIN_DAT_PERC, 2))+1])
train_set_loader = torch.utils.data.DataLoader(train_set, batch_size=BS, shuffle=True)
test_set_loader = torch.utils.data.DataLoader(test_set, batch_size=BS, shuffle=True)

In [17]:
"""
import matplotlib.pyplot as plt
i, l = next(iter(train_set_loader))
print(i.shape, i[0].shape)
sample = i[0][2]
print(sample)
plt.imshow(i[0][2].reshape(IM_HEIGHT, IM_WIDTH))
plt.show()
print(i[0][0])
print(torch.min(i[0][0]))
print(torch.max(i[0][0]))
"""

'\nimport matplotlib.pyplot as plt\ni, l = next(iter(train_set_loader))\nprint(i.shape, i[0].shape)\nsample = i[0][2]\nprint(sample)\nplt.imshow(i[0][2].reshape(IM_HEIGHT, IM_WIDTH))\nplt.show()\nprint(i[0][0])\nprint(torch.min(i[0][0]))\nprint(torch.max(i[0][0]))\n'

In [18]:
def preprocess(x, y):
    return x.to(dev), y.to(dev)
review = lambda x_b: x_b.view(len(x_b), -1)

if __name__ == '__main__':
    dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(dev)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    model = someNet()
    model.to(dev)
    loss_fn = nn.CrossEntropyLoss().to(dev)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    for epoch in range(N_EPOCH):
        train(model, dev, train_set_loader, test_set_loader, optimizer, epoch, WAND, LOG_INTERVAL, VAL_INTERVAL)


cuda


  0%|          | 0/307 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


loss: 2.308770179748535
0 tensor(0.0361, device='cuda:0', grad_fn=<DivBackward0>)



  1%|          | 1/103 [00:01<03:04,  1.81s/it]

i: 0, loss_final: 8.101900100708008, top_1_acc: 0.078125


...

i: 0, loss_final: 0.03586677461862564, top_1_acc: 0.140625



  2%|▏         | 2/103 [00:01<01:24,  1.19it/s]

i: 1, loss_final: 0.035804495215415955, top_1_acc: 0.140625


 78%|███████▊  | 240/307 [16:35<04:37,  4.15s/it]


KeyboardInterrupt: ignored

In [ ]:
test(model, dev, test_set_loader, WAND)

In [ ]:
model_name = run_name+'.pt'
MODEL_PATH = os.path.join(os.getcwd(), str(N_EPOCH))
torch.save(model.state_dict(), model_name)
print(model_name)
!ls -lah {model_name}

In [ ]:
INCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
DOWNLOAD = True
if INCOLAB and DOWNLOAD:
    from google.colab import files
    files.download(model_name)